# Example of Tucker Riemaniannian optimization 

In [1]:
# we have to set env variable to use propack, if we want to use sparse2tuck
import os
os.environ["SCIPY_USE_PROPACK"] = "1"
import torch
import numpy as np
import tucker_riemopt
from tucker_riemopt import backend as back
from tucker_riemopt import set_backend
from tucker_riemopt.tucker.tucker import Tucker
import tucker_riemopt.tucker.riemannian as riemann

###### Basic working tensors 
* $\text{Omega_dense} \approx I(0,1) $ is dense mask tensor 
* $\text{Omega_tucker}$ is Tucker representation of mask tensor
* $\text{X_dense} \approx R(0,1)$ is first target tensor approximation (dense tensor)
* $\text{X_tucker}$ is Tucker representation of first approximation

###### Examples of target tensors
* $\text{Sin_dense}(i_1,\ldots,i_d) = sin(i_1 + \ldots + i_d)$ is 2-rank target tensor 
* $\text{Sin_tucker}$ is Tucker representation of Sinus tensor
* $\text{Hilbert_dense}(i_1,\ldots,i_d) = \frac{1}{i_1 + \ldots + i_d + d}$ is target tensor with exponentially decaying singular numbers
* $\text{Hilbert_tucker}$ is Tucker representation of Hilbert tensor

In [35]:
#init tensors and mask
size = 128
set_backend("pytorch")

Sin_dense     = back.zeros([size,size,size])
# Hilbert_dense = back.zeros([size,size,size])
Omega_dense   = back.zeros([size,size,size])
X_dense       = back.zeros([size,size,size])

for i in range(size):
    for j in range(size):
        for k in range(size):
            Sin_dense[i][j][k]     = np.sin(i + j + k)
#             Hilbert_dense[i][j][k] = 1 / (i + j + k + 3)
            Omega_dense[i][j][k]   = 1 if np.random.uniform() < 0.01 else 0
            X_dense[i][j][k]       = np.random.uniform()

Omega_tucker = Tucker.from_dense(Omega_dense)
Sin_tucker   = Tucker.from_dense(Sin_dense)
X_tucker     = Tucker.from_dense(X_dense)

In [3]:
def Omega_projection(dense_tensor, Omega = Omega_dense):
    #Projection onto known tensor elements, given by dense Omega tensor
    #elementwise product
    return dense_tensor * Omega

def Tucker_Omega_projection(tucker_tensor, Omega = Omega_tucker):
    #Projection tensor given by tucker-format onto known tensor elements, given by dense Omega tensor
    #elementwise product in Tucker format
    return Tucker.from_dense(Omega_projection(tucker_tensor.to_dense()))

def Euclidean_grad(X, Target_tensor = Sin_tucker):
    return Tucker_Omega_projection(Target_tensor) - Tucker_Omega_projection(X)

def f(X, A = Sin_tucker):
    return 1/2 * (Tucker_Omega_projection(A) - Tucker_Omega_projection(X)).norm() ** 2

def line_search(eta, X):
    proj_eta = Tucker_Omega_projection(eta)
    return proj_eta.flat_inner(Euclidean_grad(X)) / (proj_eta.norm() ** 2)

def retraction(X, xi, r):
    return (X + xi).round([r, r, r])


In [4]:
xi,fx    = riemann.grad(f, X_tucker)
xi       = xi.construct()
eta      = -xi
alpha    = line_search(eta, X_tucker)
X_tucker = retraction(X_tucker, alpha * eta, 2)

In [173]:
Tucker()

In [5]:
max_iter = 150

for k in range(max_iter):
    betta = 1/(xi).norm()
    xi,fx = riemann.grad(f, X_tucker)
    xi = xi.construct()
    betta *=(xi).norm()
    eta   = -xi + betta*riemann.project(X_tucker, eta).construct()
    alpha = line_search(eta, X_tucker)
    X_tucker     = retraction(X_tucker, alpha*eta, 2)
    eps = back.sqrt(2*fx) / Sin_tucker.norm()
    
    if k%50 ==0:
        print(round(float(eps),5))
    if eps <1e-2:
        break

0.72203
0.08397
0.05961


In [6]:
print("Relative error: ||A-X||/||A|| ",round(float((Sin_dense - X_tucker.to_dense()).norm()/Sin_dense.norm()),5))

Relative error: ||A-X||/||A||  0.06931


## applied projection to each rank 

In [36]:
def get_triplet(core,v1,v2,v3, r1,r2,r3,Omega):
    return core[r1][r2][r3]*back.einsum("ijk,i,j,k->ijk",Omega, v1[:,r1],v2[:,r2],v3[:,r3])
def proj(tucker_tensor,R1,R2,R3, Omega=Omega_dense):
    res = back.zeros(tucker_tensor.shape)
    for r1 in range(R1):
        for r2 in range(R2):
            for r3 in range(R3):
                res += get_triplet(tucker_tensor.core,*tucker_tensor.factors, r1, r2, r3, Omega_dense)
    return Tucker.from_dense(res)

In [44]:
%time _=Tucker_Omega_projection(Sin_tucker)

CPU times: user 6.17 s, sys: 1.06 s, total: 7.23 s
Wall time: 975 ms


In [45]:
%time _=proj(Sin_tucker,2,2,2)

CPU times: user 6.91 s, sys: 1.76 s, total: 8.67 s
Wall time: 1.2 s


## Create list of sparse matrix

In [208]:
#Tried to use sparseTensor, but get bad results
three = []
size = 128
nnz = 300
Omega =back.zeros([size,size,size],dtype=int)
for i in range(nnz):
    three.append(tuple(np.random.randint(0, size,(3))))
    Omega[three[-1][0]][three[-1][1]][three[-1][2]]=1

In [209]:
def sp_pr(Sin_dense, three):    
    ZERO = back.zeros(Sin_dense.shape)
    vals = []
    for ind in three:
        ZERO[ind[0]][ind[1]][ind[2]] = Sin_dense[ind[0]][ind[1]][ind[2]]
        vals.append(Sin_dense[ind[0]][ind[1]][ind[2]])
    return ZERO, vals

In [270]:
def q(three,sparse_dense):
#     sparse_tensor = tucker_riemopt.SparseTensor([128,128,128],np.array(three).T,vals)
    sparse_tensor = tucker_riemopt.SparseTensor.dense2sparse(sparse_dense)
    r=Tucker.sparse2tuck(sparse_tensor,[128,128,128])
    return r

In [271]:
sparse_dense, vals=sp_pr(Sin_dense, three)

In [272]:
%time tensor = q(three,sparse_dense)

CPU times: user 1min 7s, sys: 10.7 s, total: 1min 18s
Wall time: 19.3 s


In [273]:
np.linalg.norm(tensor.to_dense() - sparse_dense)

1.913583e-05

In [274]:
np.linalg.norm(sparse_dense)

12.420635

In [164]:
#made just for fun
def hadamard_product(t1,t2):
    core = back.kron(t1.core, t2.core)
    factor0 = back.khatri_rao([t1.factors[0], t2.factors[0]]).T
    factor1 = back.khatri_rao([t1.factors[1], t2.factors[1]]).T
    factor2 = back.khatri_rao([t1.factors[2], t2.factors[2]]).T
    return Tucker(core, [factor0,factor1,factor2])